<p>
    Sources: https://products.groupdocs.app/conversion/mbox-to-csv<br>
    Sources: https://takeout.google.com/settings/takeout/downloads?pli=1
</p>


```python
# control/experiment group membership computation
import numpy as np
import pandas as pd

group = ['control', 'no-onboarding', 'DIY-onboarding']

file = '~/Downloads/Copy_of_(#10310)_Maintenance_Experiments_2021_02_05.csv'

df = pd.read_csv(file)
df['GROUP'] = None

group_assignment = np.random.choice(group, df.shape[0], p=[.85, .075, .075])
# opt for non-uniform distribution because 
# we don't want to over expose potentially poor onboarding experience 
# for risk of experiements infact degrading long-term product attach or engagement

for ix in df.index:
    df.at[ix, 'GROUP'] = group_assignment[ix]

df.to_csv('~/Downloads/group_assignment.csv', index=False)
df['GROUP'].value_counts()
"""
control           1651
no-onboarding      157
DIY-onboarding     152
"""
```


In [100]:
from csv import reader
import pandas as pd
import re
import os

In [99]:
directory = '/Users/bradley.webb/Downloads/converted/'
file_count = 0
for entry in os.scandir(directory):
    file_count += 1

In [94]:
data_fields = ['sent', 'name', 'email', 'user role', 'vhost', 'link']
core_df = pd.DataFrame(columns=data_fields, index=range(file_count))

In [102]:
group_file = '~/Downloads/group_assignment.csv'

In [103]:
group_df = pd.read_csv(group_file)

In [104]:
group_df

,VHOST,GROUP
0,aaimspm,control
1,agrentals,no-onboarding
2,alpinepropmgmt,control
3,andersoncrain,control
4,apg,no-onboarding
...,...,...
1955,remaxpeninsula,no-onboarding
1956,rentfrompeak,control
1957,rpmok001,control
1958,russellhosnerllc,DIY-onboarding


In [84]:
from_match = re.compile('from.*?')

In [97]:
ix = 0
for entry in os.scandir(directory):
    with open(entry, 'r') as read_obj:
        csv_reader = reader(read_obj)
        for row in csv_reader:
            for field in data_fields:
                re_match = re.compile(field+'.*?')
                if re_match.match(row[0].lower()):
                    data_length = len(row[0]) - len(field) - len(':')
                    datum = row[0][-data_length:].lstrip(' ')
                    core_df.at[ix, field] = datum
    ix += 1

In [108]:
core_df = core_df.join(group_df.set_index('VHOST'), on='vhost')

In [109]:
core_df.to_csv('~/Downloads/emails_received.csv', index=False)